# Functions, conditionals, loops, and intro to numerica methods
1. Functions 
2. Loops
3. Conditionals
4. Numerical methods

## Functions


In [2]:
function f1(a, b)
    return a * b
end

function f2(a, b)
    a * b
end

a = 1; b = 2
f1(a,b) ≈ f2(a,b) 

true

In [4]:
using BenchmarkTools
@btime f1(a,b)
@btime f2(a,b)

  12.012 ns (0 allocations: 0 bytes)
  11.912 ns (0 allocations: 0 bytes)


2

In [7]:
N = 100
a = range(0,100, N)
b = randn(N)

c = f1(a,b) #Error

LoadError: MethodError: no method matching *(::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, ::Vector{Float64})

[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m)
[0m[90m   @[39m [90mBase[39m [90m[4moperators.jl:587[24m[39m
[0m  *([91m::LinearAlgebra.LQ{TA, S, C} where {S<:AbstractMatrix{TA}, C<:AbstractVector{TA}}[39m, ::AbstractVecOrMat{TB}) where {TA, TB}
[0m[90m   @[39m [32mLinearAlgebra[39m [90mC:\Users\felix\AppData\Local\Programs\Julia-1.10.0\share\julia\stdlib\v1.10\LinearAlgebra\src\[39m[90m[4mlq.jl:167[24m[39m
[0m  *([91m::Number[39m, ::AbstractArray)
[0m[90m   @[39m [90mBase[39m [90m[4marraymath.jl:21[24m[39m
[0m  ...


In [10]:
function f1(a, b)
    return a .* b
end

N = 100
a = range(0,100, N)
b = randn(N)

@btime c = f1(a,b);

  154.375 ns (1 allocation: 896 bytes)


In [19]:
function f1!(out, a, b) #a,b->c
     out .= a .* b
end

N = 100
a = range(0,100, N)
b = randn(N)

out = similar(a)
@btime f1!(out, a, b);

  139.394 ns (0 allocations: 0 bytes)


In [30]:
function f3(a::Vector{Float64}, b::Vector{Float64})
    return a .* b
end

c = f3(1.0, 1.0) #Error 

LoadError: MethodError: no method matching f3(::Float64, ::Float64)

In [39]:
f(x) = sin(1 / x)

f (generic function with 1 method)

In [42]:
map(x -> sin(1 / x), randn(3));  # apply function to each element

In [41]:
f(x, a = 1) = exp(cos(a * x))

f (generic function with 2 methods)

### Exercises
1. Create a function out-place of $f(x,y)=(a-x)^2 + b(y)^2$
2. Create a function in-place of $f(x,y)=(a-x)^2 + b(y)^2$
3. Create a grid of x and y of dimension N=100 and evaluate the functions. 
4. Test if the results are the same ($\approx$) and the performance (@btime and @time). 

## Loops

In [43]:
for i in 1:10
    print(i)
end


12345678910

In [44]:
index = range(1,100)
for i in index
    print(i)
end

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100

In [51]:
x = rand(100)
k_x = Any[]
v_x = Any[]

for (k,v) in enumerate(x)
    push!(k_x, k)
    push!(v_x, v)
end

In [53]:
# Crear matrices A y B
A = rand(100, 10)  # Matriz de 100x10 con valores aleatorios
B = rand(10, 100)  # Matriz de 10x100 con valores aleatorios

# Multiplicación directa de matrices
C1 = A * B;  # Resultado es una matriz de 100x100


In [56]:
# Inicializar matriz C con ceros
C2 = zeros(100, 100)

# Multiplicar matrices usando un loop
for i in 1:100
    for j in 1:100
        for k in 1:10
            C2[i, j] += A[i, k] * B[k, j]
        end
    end
end

C1 ≈ C2

true

In [ ]:
using Plots
# Iterates a function from an initial condition 
function iterate_map(f, x0, T)
    x = zeros(T + 1)
    x[1] = x0
    for t in 2:(T + 1)
        x[t] = f(x[t - 1])
    end
    return x
end

function ts_plot(f, x0, T; xlabel = "t", label = "k_t")
    x = iterate_map(f, x0, T)
    plot(0:T, x; xlabel, label)
    plot!(0:T, x; seriestype = :scatter, mc = :blue, alpha = 0.7, label = nothing)
end


p = (A = 2, s = 0.3, alpha = 0.3, delta = 0.4, xmin = 0, xmax = 4)
g(k; p) = p.A * p.s * k^p.alpha + (1 - p.delta) * k
k0 = 0.25
ts_plot(k -> g(k; p), k0, 5)

## Conditionals

Consider the equation $v = p+\beta v$ where $p,\beta$ are given, and $v$ is a scalar to solve. 

*This simple example can be solve like* $v=p/(1-\beta)$. 

However, we are going to try $v=f(v):=p+\beta v$ 

### While loops

In [58]:
using LinearAlgebra
# poor style
p = 1.0 
beta = 0.9
maxiter = 1000
tolerance = 1.0E-7
v_iv = 0.8 # initial condition

# setup the algorithm
v_old = v_iv
normdiff = Inf
iter = 1
while normdiff > tolerance && iter <= maxiter
    v_new = p + beta * v_old # the f(v) map
    normdiff = norm(v_new - v_old)

    # replace and continue
    v_old = v_new
    iter = iter + 1
end
println("Fixed point = $v_old
  |f(x) - x| = $normdiff in $iter iterations")

Fixed point = 9.999999173706609
  |f(x) - x| = 9.181037796679448e-8 in 155 iterations


### For and if

In [59]:
# setup the algorithm
v_old = v_iv
normdiff = Inf
iter = 1
for i in 1:maxiter
    v_new = p + beta * v_old # the f(v) map
    normdiff = norm(v_new - v_old)
    if normdiff < tolerance # check convergence
        iter = i
        break # converged, exit loop
    end
    # replace and continue
    v_old = v_new
end
println("Fixed point = $v_old
  |f(x) - x| = $normdiff in $iter iterations")

Fixed point = 9.999999081896231
  |f(x) - x| = 9.181037796679448e-8 in 154 iterations


## Excercises
1. Put the examples in functions, test the performance. 
2. Check if the functions have the same result and are the same that analytical solution $v=p/(1-\beta)$

## Numerical methods
$f(x,y)=(a-x)^2 + b(y-x^2)^2$

### Solving non-linear systems

In [ ]:
using NLsolve

function f!(F, x)
    F[1] = x[1]^2 + 2x[2]^2 - 1
    F[2] = 2x[1]^2 + x[2]^2 - 1
end

x = nlsolve(f!, [ 0.1; 1.2], autodiff = :forward)
print(x.zero)

### Nonlinear Optimization

In [62]:
using Optim

f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv) # i.e. optimize(f, x_iv, NelderMead())

 * Status: success

 * Candidate solution
    Final objective value:     3.525527e-09

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    60
    f(x) calls:    117


In [63]:
using Optimization, OptimizationOptimJL

rosenbrock(u,p) =  (p[1] - u[1])^2 + p[2] * (u[2] - u[1]^2)^2
u0 = zeros(2)
p  = [1.0,100.0]

prob = OptimizationProblem(rosenbrock,u0,p)
sol = solve(prob,NelderMead())

retcode: Success
u: 2-element Vector{Float64}:
 0.9999634355313174
 0.9999315506115275

## Exercises
1. Compare the results of the three methods
2. Plot the 3D plane of f(x,y),x,y. 
3. Add to the plot the solution of the 3 methods. 